<a href="https://colab.research.google.com/github/aicuai/GenAI-Steam/blob/main/AICU_gpt4o_transribe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🎙️ GPT-4o Transcribe
### 自分で作るオンライン会議文字起こし

Google Drive 上の Zoom/Meet 録画ファイル (mp4) を分割し、OpenAI GPT-4o-mini-transcribe を用いて文字起こしします。

このコードへのリンク [j.aicu.ai/gpt4oTransc](https://j.aicu.ai/gpt4oTransc)

解説は「サクリ！AIツール: 自分で作るオンライン会議文字起こし」 - AICU Japan 株式会社
[https://note.com/aicu](https://note.com/aicu/m/m8c3003efc9f8)

(C) 2025 Akihiko SHIRAI - AICU Japan - 著作権は放棄していません

### 🔐 Google Colab シークレット設定マニュアル

このセルでは、Google Colab に保存された `OPENAI_API_KEY` を使って OpenAI API の接続テストを行います。

#### ✅ 事前準備
以下の手順で `OPENAI_API_KEY` を Colab に設定してください。

1. 画面左のサイドバーにある「鍵マーク（🔐）」をクリック（「環境設定」→「シークレットを管理」）
2. 「+ 新しいシークレットを追加」をクリック
3. 以下のように入力して「保存」：
   - **名前（Name）**: `OPENAI_API_KEY`
   - **値（Value）**: `sk-...` から始まるあなたのOpenAI APIキー

> ⚠️ これにより、コード内にハードコードせずに安全にAPIキーを扱うことができます。

---

設定後にこのセルを実行することで、APIキーの有効性や利用可能なモデル一覧が確認できます。

In [ ]:
# @title
# ✅ セットアップ（初回のみ）
!pip install openai ffmpeg-python --quiet
!apt-get install -y ffmpeg > /dev/null
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# @title
# 🔍 OpenAI API キーの動作確認（Colab secrets使用）
import openai
from google.colab import userdata

api_key = userdata.get('OPENAI_API_KEY')
if not api_key:
    raise ValueError("❌ 'OPENAI_API_KEY' が Colab のシークレットに設定されていません")

openai.api_key = api_key

try:
    models = openai.models.list()
    model_ids = [m.id for m in models.data]
    print("✅ OpenAI API キーは有効です")
    print("利用可能なモデル例:", ", ".join(model_ids[:5]), "...")
except Exception as e:
    print("❌ OpenAI API キーが無効または接続エラーです")
    print("エラー内容:", e)

## 🔧 Step 1: Zoom録画の設定と対象ファイルの決定

このセルでは以下の3つの入力を行います：

- **PROMPT**: 文字起こし時にAIへ与える指示（例：「逐語的に」「要約せずに」など）
- **DIR**: Google Drive上の録画フォルダパス（例：/content/drive/MyDrive/Meet Recordings）
- **FILENAME**: 対象となるファイル名（拡張子 `.mp4` は省略可）

🎯 ファイル名が省略された場合は、フォルダ内の最初の `.mp4` または拡張子なしファイルが自動的に選択されます。

---

📌 ファイルが存在しない場合：
1. `.mp4` 拡張子を補って再確認
2. それでも見つからなければ `FileNotFoundError` を出して処理を停止します

このセルの実行に成功すると、次のステップで mp3 変換・分割・文字起こしが実行可能になります。

In [ ]:
# 📋 文字起こしの設定フォーム
# @title 📋 Zoom文字起こしの基本設定
PROMPT = "これはZoom会議の議事録です。逐語的に起こしてください。"  # @param {type:"string"}
DIR = "/content/drive/MyDrive/Meet Recordings"  # @param {type:"string"}
FILENAME = "MeetRecFile(mp4)"  # @param {type:"string"}

from pathlib import Path

# 📂 入力パスの処理
DIR = Path(DIR).expanduser()
PROMPT = PROMPT.strip()
FILENAME = FILENAME.strip()

# 🎯 対象ファイルの決定
files = sorted([f for f in DIR.iterdir() if f.suffix == '.mp4' or '.' not in f.name])
if not files:
    raise FileNotFoundError(f"{DIR} に .mp4 または拡張子なしのファイルが見つかりません")

# ユーザー指定 or 最初のファイル
target_file_stem = FILENAME if FILENAME else files[0].stem
target_file = DIR / target_file_stem
input_path = target_file

# .mp4 を補完して再確認
if not input_path.exists():
    target_file = DIR / f"{target_file_stem}.mp4"
    input_path = target_file
    if not input_path.exists():
        raise FileNotFoundError(f"対象ファイルが見つかりません: {target_file_stem} または {target_file_stem}.mp4")

print(f"✅ 処理対象: {input_path.name}")

In [ ]:
# @title
# 🧠 実行コード（分割前・改良版）
import os
import ffmpeg
import time
from datetime import datetime
from pathlib import Path
import openai
from google.colab import userdata

# 🔑 OpenAI APIキー取得（Google Colabのシークレットから）
openai.api_key = userdata.get("OPENAI_API_KEY")
if not openai.api_key:
    raise ValueError("❌ OpenAI APIキーが見つかりません。メニュー「ランタイム > シークレットを管理」で `OPENAI_API_KEY` を追加してください。")

# 📋 入力パラメータ
DIR = Path(DIR)  # フォルダ（@paramで指定）
PROMPT = PROMPT.strip()
FILENAME = FILENAME.strip()

# 🎯 対象ファイル決定
files = sorted([f for f in DIR.iterdir() if f.suffix == '.mp4' or '.' not in f.name])
if not files:
    raise FileNotFoundError(f"{DIR} に処理対象ファイルが見つかりません")

target_file_stem = FILENAME if FILENAME else files[0].stem
target_file = DIR / f"{target_file_stem}"
input_path = target_file

# 🔁 拡張子なしなら.mp4を付けて再試行
if not input_path.exists():
    target_file = DIR / f"{target_file_stem}.mp4"
    input_path = target_file
    if not input_path.exists():
        raise FileNotFoundError(f"対象ファイルが見つかりません: {target_file_stem} または {target_file_stem}.mp4")

print(f"🎯 処理対象: {input_path.name}")

base = input_path.stem
mp3_path = DIR / f"{base}.mp3"
segment_dir = DIR / "mp3segment"
segment_dir.mkdir(exist_ok=True)

# 🎞️ mp4 → mp3 変換
print(f"🎞️ 変換: {input_path.name} → {mp3_path.name}")
ffmpeg.input(str(input_path)).output(
    str(mp3_path),
    vn=None, ar=44100, ac=2, **{'b:a': '128k'}
).overwrite_output().run()

# ⏱️ 音声長取得
duration = float(ffmpeg.probe(str(mp3_path))['format']['duration'])
print(f"⏱️ 長さ: {duration:.1f} 秒")

# ✂️ 分割
print("✂️ 分割中...")
seg_fmt = segment_dir / f"{base}_%03d.mp3"
ffmpeg.input(str(mp3_path)).output(
    str(seg_fmt),
    f='segment', segment_time=1450, c='copy'
).overwrite_output().run()

# 🎧 セグメント取得
segments = sorted(segment_dir.glob("*.mp3"))

# 📝 書き起こし
output_txt = DIR / f"{base}.txt"
with open(output_txt, 'w', encoding='utf-8') as out:
    out.write(f"=== GPT-4o-mini Transcribe: {target_file} ===\n\n")
    for i, seg_path in enumerate(segments):
        print(f"🎙️ Segment {i+1}/{len(segments)}: {seg_path.name}")
        with seg_path.open("rb") as f:
            try:
                t0 = time.time()
                result = openai.audio.transcriptions.create(
                    model="gpt-4o-mini-transcribe",
                    file=f,
                    language="ja",
                    prompt=PROMPT,
                    temperature=0.2
                )
                dt = time.time() - t0
                out.write(f"[{seg_path.name}] ({dt:.1f} 秒)\n{result.text}\n\n")
            except Exception as e:
                out.write(f"[{seg_path.name}] エラー: {str(e)}\n\n")

# 🗂️ 終了処理
done_dir = DIR / "done"
done_dir.mkdir(exist_ok=True)
input_path.rename(done_dir / input_path.name)

# 🧹 後始末
for f in segment_dir.glob("*.mp3"):
    f.unlink()
segment_dir.rmdir()

print("✅ 完了")